In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import igraph as ig
import os
import sys
from datetime import datetime, timedelta
import networkx as nx

In [41]:
# import datasets
all = pd.read_csv("./data/all.csv")
all.head()

/var/folders/n0/r52jwj117lx5t69xm0bw9b980000gn/T/ipykernel_11318/896371191.py:2: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  all = pd.read_csv("./data/all.csv")


,end_lat,end_lng,end_station_id,end_station_name,ended_at,member_casual,ride_id,rideable_type,start_lat,start_lng,start_station_id,start_station_name,started_at,distance
0,41.9322,-87.6586,152.0,Lincoln Ave & Diversey Pkwy,2020-04-26 18:12:03,member,A847FADBBC638E45,docked_bike,41.8964,-87.6610,86,Eckhart Park,2020-04-26 17:45:14,3.981346
1,41.9306,-87.7238,499.0,Kosciuszko Park,2020-04-17 17:17:03,member,5405B80E996FF60D,docked_bike,41.9244,-87.7154,503,Drake Ave & Fullerton Ave,2020-04-17 17:08:54,0.979629
2,41.8679,-87.6230,255.0,Indiana Ave & Roosevelt Rd,2020-04-01 18:08:36,member,5DD24A79A4E006F4,docked_bike,41.8945,-87.6179,142,McClurg Ct & Erie St,2020-04-01 17:54:13,2.984661
3,41.8992,-87.6722,657.0,Wood St & Augusta Blvd,2020-04-07 13:02:31,member,2A59BBDF5CDBA725,docked_bike,41.9030,-87.6975,216,California Ave & Division St,2020-04-07 12:50:19,2.141375
4,41.9695,-87.6547,323.0,Sheridan Rd & Lawrence Ave,2020-04-18 11:15:54,casual,27AD306C119C6158,docked_bike,41.8902,-87.6262,125,Rush St & Hubbard St,2020-04-18 10:22:59,9.119685


In [43]:
# Get the node_list, edge_list and timestamp
def return_nm(df):
    df['started_at'] = pd.to_datetime(df['started_at'])
    df['ended_at'] = pd.to_datetime(df['ended_at'])
    # started_atで昇順にソート
    df = df.sort_values(by='started_at')
    current_timestamp = df['started_at'].min()
    end_timestamp = df["started_at"].max()
    interval = timedelta(days=1)

    node_list = []
    edge_list = []
    timestamp = []

    while current_timestamp <= end_timestamp:
        timestamp.append(current_timestamp)
        next_timestamp = current_timestamp + interval
        df_tmp = df[(df['started_at'] >= current_timestamp) & (df['started_at'] < next_timestamp)]

        # node_listにはユニークなstart_station_idを追加
        node_list.append(len(df_tmp['start_station_id'].unique()))

        # edge_listにはdf_tmpの行数を追加
        edge_list.append(len(df_tmp))

        current_timestamp = next_timestamp

    return node_list, edge_list, timestamp

In [68]:
all.head()

,end_lat,end_lng,end_station_id,end_station_name,ended_at,member_casual,ride_id,rideable_type,start_lat,start_lng,start_station_id,start_station_name,started_at,distance
0,41.9322,-87.6586,152.0,Lincoln Ave & Diversey Pkwy,2020-04-26 18:12:03,member,A847FADBBC638E45,docked_bike,41.8964,-87.6610,86,Eckhart Park,2020-04-26 17:45:14,3.981346
1,41.9306,-87.7238,499.0,Kosciuszko Park,2020-04-17 17:17:03,member,5405B80E996FF60D,docked_bike,41.9244,-87.7154,503,Drake Ave & Fullerton Ave,2020-04-17 17:08:54,0.979629
2,41.8679,-87.6230,255.0,Indiana Ave & Roosevelt Rd,2020-04-01 18:08:36,member,5DD24A79A4E006F4,docked_bike,41.8945,-87.6179,142,McClurg Ct & Erie St,2020-04-01 17:54:13,2.984661
3,41.8992,-87.6722,657.0,Wood St & Augusta Blvd,2020-04-07 13:02:31,member,2A59BBDF5CDBA725,docked_bike,41.9030,-87.6975,216,California Ave & Division St,2020-04-07 12:50:19,2.141375
4,41.9695,-87.6547,323.0,Sheridan Rd & Lawrence Ave,2020-04-18 11:15:54,casual,27AD306C119C6158,docked_bike,41.8902,-87.6262,125,Rush St & Hubbard St,2020-04-18 10:22:59,9.119685


In [87]:
# allからstart_station_idとend_station_idを取り出す
all_station = all[['start_station_id', 'end_station_id']]
# 欠損値を削除
all_station = all_station.dropna()
all_station.head()



,start_station_id,end_station_id
0,86,152.0
1,503,499.0
2,142,255.0
3,216,657.0
4,125,323.0


In [89]:
# end_station_idの.0をstrにして削除
all_station['end_station_id'] = all_station['end_station_id'].astype(str)
all_station['end_station_id'] = all_station['end_station_id'].str.replace('.0', '')
# start_station_idの.0をstrにして削除
all_station['start_station_id'] = all_station['start_station_id'].astype(str)
all_station['start_station_id'] = all_station['start_station_id'].str.replace('.0', '')

In [92]:
# all_stationで隣接行列を作成
adjacency_matrix = pd.crosstab(all_station['start_station_id'], all_station['end_station_id'])
adjacency_matrix.head()

end_station_id,100,101,102,103,106,107,108,109,11,110,...,TA1309000058,TA1309000059,TA1309000061,TA1309000063,TA1309000064,TA1309000066,TA1309000067,WL-008,WL-011,WL-012
start_station_id,,,,,,,,,,,,,,,,,,,,,
100,447,0,0,0,38,33,2,2,0,131,...,0,0,0,0,0,0,0,0,0,0
101,0,1103,11,0,0,4,0,0,12,0,...,0,0,0,0,0,0,0,0,0,0
102,0,10,210,0,0,0,0,0,10,0,...,0,0,0,0,0,0,0,0,0,0
103,2,0,0,39,0,8,1,8,0,6,...,0,0,0,0,0,0,0,0,0,0
106,29,1,0,0,849,18,1,0,0,88,...,0,0,0,0,0,0,0,0,0,0


In [99]:
g = nx.from_pandas_edgelist(all_station, source='start_station_id', target='end_station_id', create_using=nx.Graph())
g = ig.Graph.from_networkx(g)


In [103]:
layout = g.layout_auto()
%pip install pycairo
ig.plot(g, layout=layout)

plt.show()

OSError: "/usr/local/bin/zsh" shell not found